Fernando Arturo Arevalo Perez  
Numero de alumno: 323018942  
12. 5. Apache Spark Streaming

### Ejercicio 1  
En este ejercicio debemos generar un proceso que realice predicciones sobre un conjunto de datos en streaming y utilizando un modelo de Machine Learning de clasificación. Para ello, deberás completar el notebook de “Ejercicio_Predicción en Streaming con Spark ML y Spark Streaming”, sustituyendo los apartados comentados ,con “##”, por el código que realice la función especificada.

# Ejercicio Práctico_Predicción en Streaming con Spark ML y Spark Streaming

En este notebook vamos a cargar un pipeline que tiene un conjunto de fases de pre-procesamiento y un modelo de clasificacion predecir la probabilidad de un paciente de sufrir un ataque al corazón. La predicción se realizará sobre datos en streaming optenidos a partir del csv de heart.csv

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import *

In [4]:
from pyspark.sql import SparkSession
## Inicia una sesion de Spark
spark = SparkSession.builder.appName('Heart desease').getOrCreate()

In [8]:
## Carga y visualiza el csv de Ejercicios\data\heart.csv con el nombre de heart
heart = spark.read.csv('data/heart.csv', 
                       inferSchema = True, 
                       header = True)
heart.show(5)

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
only showing top 5 rows



In [9]:
schema = StructType( \
                     [StructField("age", LongType(),True), \
                      StructField("sex", LongType(), True), \
                      StructField("cp", LongType(), True), \
                      StructField('trestbps', LongType(), True), \
                      StructField("chol", LongType(), True), \
                      StructField("fbs", LongType(), True), \
                      StructField("restecg", LongType(), True), \
                      StructField("thalach", LongType(), True),\
                      StructField("exang", LongType(), True), \
                      StructField("oldpeak", DoubleType(), True), \
                      StructField("slope", LongType(),True), \
                      StructField("ca", LongType(), True), \
                      StructField("thal", LongType(), True), \
                      StructField("target", LongType(), True), \
                        ])

In [10]:
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType,StructField,LongType, StringType,DoubleType,TimestampType


df = heart.withColumnRenamed("target","label")
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trestbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalach: integer (nullable = true)
 |-- exang: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: integer (nullable = true)
 |-- ca: integer (nullable = true)
 |-- thal: integer (nullable = true)
 |-- label: integer (nullable = true)



In [28]:
testDF, trainDF = df.randomSplit([0.3, 0.7])

### Carga del Pipeline

In [26]:
from pyspark.ml import PipelineModel

In [32]:
#Cargamos los pipelines
pModel = PipelineModel.load("/Pipelines")

pModel

In [2]:
## Comprueba que el pipeline anterior funciona correctamente. Para ello realiza una prediccion sobre el conjunto de 
## datos de trainDF y muestra la prediccion
prediccion = pipelaine.transform(trainDF)

prediccion.show()

In [81]:
testData = testDF.repartition(10)

testData.write.format("CSV").option("header",False).save("/heart_streaming/")

## Creando predicciones en Streaming

In [82]:
## Utiliza los csv guardados en data/heart_streaming para simular un proceso de datos en streaming.
## Para ello, utiliza la funcion de spark.readStream 
## En la configuración pon: que se importe un archivo por ejecucion
## que se renombre la variable de "output"a "label"
## Llama a este proceso con el nombre sourceStream
dfss = df.schema
stream = spark.readStream.schema(dfss).option("maxFilesPerTrigger", 1).csv("/heart_streaming")
sourceStream = stream.withColumnRenamed("output", "label")

In [83]:
## Utiliza el pipeline "pModel" para realizar predicciones utilizando los datos en streaming de "sourceStream"
## De la predicción selecciona las variables label, probability, prediction. 
## Llama a este proceso con el nombre de "prediction1"
prediction1 = pModel.transform(sourceStream)
prediction1 = prediction1.select("label", "probability", "prediction")

In [84]:
display(prediction1)

DataFrame[label: bigint, probability: vector, prediction: double]

#### Mostrando las predicciones en consola

In [85]:
## Obten las predicciones sobre los datos en streaming, para ello utiliza prediction1.writeStream. En las opciones de
## configuracion pon: "format" igual a "console" 
## en .trigger igual (once=True),
query = (
    prediction1
    .writeStream
    .outputMode("append")
    .format("console")
    .trigger(once = True)
    .queryName("prediction1")
    .start()
)

## y permite que el proceso espere hasta que se complete con .awaitTermination()
query.awaitTermination()

#### Guardando las perdicciones en Memoria

In [3]:
## Obten las predicciones sobre los datos en streaming, para ello utiliza prediction1.writeStream. 
## En las opciones de configuracion pon: que los resultados se guarden en memoria, 
## que el .outputMode sea "append"
## que el nombre de la query "queryName" sea "prediction4"

query = (
    prediction1
    .writeStream
    .outputMode("append")
    .format("memory")
    .trigger(once = True)
    .queryName("prediction4")
    .start()
)

In [88]:
for x in range(2):
    df = spark.sql(
        "SELECT * FROM prediction4")
    df.show(10)
df

+-----+--------------------+----------+
|label|         probability|prediction|
+-----+--------------------+----------+
|    1|[0.04086978924170...|       1.0|
|    0|[0.98184892212735...|       0.0|
|    1|[0.00474279761632...|       1.0|
|    1|[0.35775366097494...|       1.0|
|    1|[0.05755909903937...|       1.0|
|    0|[0.95305536703752...|       0.0|
|    0|[0.94079962605713...|       0.0|
|    0|[0.13017480179914...|       1.0|
|    0|[0.99807916786174...|       0.0|
|    1|[0.15541832735450...|       1.0|
+-----+--------------------+----------+
only showing top 10 rows

+-----+--------------------+----------+
|label|         probability|prediction|
+-----+--------------------+----------+
|    1|[0.04086978924170...|       1.0|
|    0|[0.98184892212735...|       0.0|
|    1|[0.00474279761632...|       1.0|
|    1|[0.35775366097494...|       1.0|
|    1|[0.05755909903937...|       1.0|
|    0|[0.95305536703752...|       0.0|
|    0|[0.94079962605713...|       0.0|
|    0|[0.1301

DataFrame[label: bigint, probability: vector, prediction: double]

In [ ]:
## Valida que el proceso de streaming está activo y después muestra el estado
print("Consulta activa: ", sourceStream.isStreaming)
print("Estado consulta: ", sourceStream.status)